# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [94]:
# Import Python packages 
import pandas as pd
import cassandra
import os
import glob
import numpy as np
import csv

#### Creating list of filepaths to process original event csv data files

In [95]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\Youssef\Documents\Data Engineer\Lesson 1\Project2


In [96]:
len(file_path_list)

30

In [97]:
# Helper Cell I created for simplicity
new_df = pd.DataFrame()
for file in file_path_list: 
    df = pd.read_csv(file)
    filtered_df = df.loc[(~df['artist'].isna()), ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']]
    new_df = pd.concat([new_df, filtered_df])
    
new_df.reset_index(drop=True, inplace=True)
new_df.to_csv('new_datafile_self_created.csv', index=False)

In [98]:
new_df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8.0
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8.0
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8.0
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8.0
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8.0


In [99]:
new_df.shape

(6820, 11)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [100]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [101]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [102]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e: 
    print("Error connection to Cassandra server on my local machine: ")
    print(e)

#### Create Keyspace

In [103]:
query = ("""CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}""")
try: 
    session.execute(query)
except Exception as e: 
    print("Error creating sparkify keyspace: ")
    print(e)

#### Set Keyspace

In [104]:
try: 
    session.set_keyspace("sparkify")
except Exception as e: 
    print("Error connection to sparkify keyspace: ")
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### Question / Query 1: 

In [105]:
# Make sure that a primary key of session_id & itemInSession would be a good choice for songs_library table (and IT IS)
new_df.loc[new_df['sessionId'] == 1076].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6693,Ryan Bingham,Rylan,M,0,George,247.30077,paid,"Birmingham-Hoover, AL",1076,Bread & Water,16.0
6698,Jason Mraz,Rylan,M,1,George,264.69832,paid,"Birmingham-Hoover, AL",1076,Summer Breeze,16.0
6701,Five Iron Frenzy,Rylan,M,2,George,236.09424,paid,"Birmingham-Hoover, AL",1076,Canada,16.0
6704,Brooklyn Bounce,Rylan,M,3,George,359.13098,paid,"Birmingham-Hoover, AL",1076,Get Ready To Bounce Recall 08,16.0
6708,The Pussycat Dolls,Rylan,M,4,George,190.95465,paid,"Birmingham-Hoover, AL",1076,Sway,16.0


##### Create `songs_library` Table 

In [106]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = ("""CREATE TABLE IF NOT EXISTS songs_library 
    (artist TEXT, song TEXT, length FLOAT, session_id INT, itemInSession INT, 
    PRIMARY KEY (session_id, itemInSession))""")
try: 
    session.execute(query1)
except Exception as e: 
    print("Error creating songs_library table: ")
    print(e)      

##### Insert Into `songs_library` Table 

In [44]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_library (artist, song, length, session_id, itemInSession)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

In [107]:
## Second Option [My Own] 
query = "INSERT INTO songs_library (artist, song, length, session_id, itemInSession)"
query = query + "VALUES (%s, %s, %s, %s, %s)"
def insert_into_songs_library(row): 
    session.execute(query, (row['artist'], row['song'], float(row['length']), int(row['sessionId']), int(row['itemInSession'])))

new_df.apply(insert_into_songs_library, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
6815    None
6816    None
6817    None
6818    None
6819    None
Length: 6820, dtype: object

##### Select from `songs_library` Table (for validation)

In [108]:
new_df.loc[(new_df['sessionId'] == 338) & (new_df['itemInSession'] == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
964,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50.0


In [109]:
select_query = ("""SELECT * FROM songs_library WHERE session_id = %s AND itemInSession = %s""")
try: 
    rows = session.execute(select_query, (338, 4))
except Exception as e: 
    print("Error selecting from songs_library table: ")
    print(e)   
    
for row in rows: 
    print(f'Artist: {row.artist}, Song: {row.song}, Song Length: {row.length}, Session_Id: {row.session_id}, Item_In_Session {row.iteminsession}')

Artist: Faithless, Song: Music Matters (Mark Knight Dub), Song Length: 495.30731201171875, Session_Id: 338, Item_In_Session 4


#### Question / Query 2

In [110]:
# Make sure that a primary key of user_id, session_id & itemInSession would be a good choice for songs_library table (and IT IS)
new_df.loc[new_df['sessionId'] == 1076].head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
6693,Ryan Bingham,Rylan,M,0,George,247.30077,paid,"Birmingham-Hoover, AL",1076,Bread & Water,16.0
6698,Jason Mraz,Rylan,M,1,George,264.69832,paid,"Birmingham-Hoover, AL",1076,Summer Breeze,16.0
6701,Five Iron Frenzy,Rylan,M,2,George,236.09424,paid,"Birmingham-Hoover, AL",1076,Canada,16.0
6704,Brooklyn Bounce,Rylan,M,3,George,359.13098,paid,"Birmingham-Hoover, AL",1076,Get Ready To Bounce Recall 08,16.0
6708,The Pussycat Dolls,Rylan,M,4,George,190.95465,paid,"Birmingham-Hoover, AL",1076,Sway,16.0


##### Create `user_songs_history` Table 

In [111]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182


query2 = ("""CREATE TABLE IF NOT EXISTS user_songs_history 
    (artist TEXT, song TEXT, itemInSession INT, user_first_name TEXT, user_last_name TEXT, user_id INT, session_id INT,
    PRIMARY KEY ((user_id, session_id), itemInSession))""")
try: 
    session.execute(query2)
except Exception as e: 
    print("Error creating user_songs_history table: ")
    print(e)      

##### Insert Into `user_songs_history` Table 

In [65]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_songs_history (artist, song, itemInSession, user_first_name, user_last_name, user_id, session_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (line[0], line[9], int(line[3]), line[1], line[4], int(line[10]), int(line[8])))
                    

In [112]:
## Second Option [My Own] 
query = "INSERT INTO user_songs_history (artist, song, itemInSession, user_first_name, user_last_name, user_id, session_id)"
query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
def insert_into_user_songs_history(row): 
    session.execute(query, (row['artist'], row['song'], int(row['itemInSession']), row['firstName'], row['lastName'], int(row['userId']), int(row['sessionId'])))

new_df.apply(insert_into_user_songs_history, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
6815    None
6816    None
6817    None
6818    None
6819    None
Length: 6820, dtype: object

##### Select from `user_songs_history` Table (for validation)

In [113]:
new_df.loc[(new_df['userId'] == 10) & (new_df['sessionId'] == 182)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
151,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10.0
152,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10.0
153,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10.0
154,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10.0


In [114]:
select_query = ("""SELECT * FROM user_songs_history WHERE user_id = %s AND session_id = %s""")
try: 
    rows = session.execute(select_query, (10, 182))
except Exception as e: 
    print("Error selecting from user_songs_history table: ")
    print(e)   
    
for row in rows: 
    print(f'Artist: {row.artist}, Song: {row.song}, First Name: {row.user_first_name}, Last Name: {row.user_last_name}, UserId: {row.user_id}, SessionId: {row.session_id}, ItemInSession: {row.iteminsession}')

Artist: Down To The Bone, Song: Keep On Keepin' On, First Name: Sylvie, Last Name: Cruz, UserId: 10, SessionId: 182, ItemInSession: 0
Artist: Three Drives, Song: Greece 2000, First Name: Sylvie, Last Name: Cruz, UserId: 10, SessionId: 182, ItemInSession: 1
Artist: Sebastien Tellier, Song: Kilometer, First Name: Sylvie, Last Name: Cruz, UserId: 10, SessionId: 182, ItemInSession: 2
Artist: Lonnie Gordon, Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit), First Name: Sylvie, Last Name: Cruz, UserId: 10, SessionId: 182, ItemInSession: 3


#### Question / Query 3

In [115]:
# Make sure that a primary key of song, user_id, session_id would be a good choice for songs_library table (and IT IS)
new_df.loc[new_df['song'] == 'All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
219,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95.0
2442,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29.0
2646,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80.0


##### Create `music_history` Table 

In [116]:
query3 = ("""CREATE TABLE IF NOT EXISTS music_history 
    (user_first_name TEXT, user_last_name TEXT, song TEXT, user_id INT, session_id INT,
    PRIMARY KEY (song, user_id, session_id))""")

try: 
    session.execute(query3)
except Exception as e: 
    print("Error creating music_history table: ")
    print(e)      

##### Insert Into `music_history` Table 

In [82]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_history (user_first_name, user_last_name, song, user_id, session_id)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (line[1], line[4], line[9], int(line[10]), int(line[8]) ))
                    

In [117]:
## Second Option [My Own] 
query = "INSERT INTO music_history (user_first_name, user_last_name, song, user_id, session_id)"
query = query + "VALUES (%s, %s, %s, %s, %s)"
def insert_into_music_history(row): 
    session.execute(query, (row['firstName'], row['lastName'], row['song'], int(row['userId']), int(row['sessionId'])))

new_df.apply(insert_into_music_history, axis=1)

0       None
1       None
2       None
3       None
4       None
        ... 
6815    None
6816    None
6817    None
6818    None
6819    None
Length: 6820, dtype: object

##### Select from `music_history` Table (for validation)

In [118]:
new_df.loc[new_df['song'] == 'All Hands Against His Own']

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
219,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95.0
2442,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29.0
2646,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80.0


In [119]:
select_query = ("""SELECT user_first_name, user_last_name FROM music_history WHERE song = %s""")
try: 
    rows = session.execute(select_query, ('All Hands Against His Own', ))
except Exception as e: 
    print("Error selecting from music_history table: ")
    print(e)   
    
for row in rows: 
    print(f'First Name: {row.user_first_name}, Last Name: {row.user_last_name}')

First Name: Jacqueline, Last Name: Lynch
First Name: Tegan, Last Name: Levine
First Name: Sara, Last Name: Johnson


### Drop the tables before closing out the sessions

In [120]:
try: 
    session.execute("DROP TABLE songs_library") 
    session.execute("DROP TABLE user_songs_history") 
    session.execute("DROP TABLE music_history") 
except Exception as e: 
    print("Error on deleting tables: ")
    print(e)   
    

### Close the session and cluster connection¶

In [121]:
session.shutdown()
cluster.shutdown()